<a href="https://colab.research.google.com/github/BrotherKim/KAIST_CODE/blob/master/SEP531/term/5_KoBERT_eval_ks_std.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_finetuned_data_ks.pt'

In [5]:
!ls -l /content/drive/MyDrive/KAIST/SEP531

total 2552619
-rw------- 1 root root   1894595 Nov  8 06:21  4차년도.csv
-rw------- 1 root root   2142130 Dec  4 08:42  4_utf8.csv
-rw------- 1 root root   2167298 Dec  4 09:52  4차년도_utf8.csv
-rw------- 1 root root   2642158 Nov  8 06:21  5차년도_2차.csv
-rw------- 1 root root   2976104 Dec  4 08:41  5_2_utf8.csv
-rw------- 1 root root   2976104 Dec  4 09:52  5차년도_2차_utf8.csv
-rw------- 1 root root   1433551 Nov  8 06:21  5차년도.csv
-rw------- 1 root root   1659315 Dec  4 08:41  5_utf8.csv
-rw------- 1 root root   1678755 Dec  4 09:52  5차년도_utf8.csv
drwx------ 2 root root      4096 Dec  9 10:54  HAK_TO_HYUNG
drwx------ 2 root root      4096 Nov 23 14:55  hw3_cp
drwx------ 2 root root      4096 Dec  6 14:20  hw4_cp
-rw------- 1 root root 368848687 Nov  8 07:19  KoBERT_emotion_finetuned_data_added.pt
-rw------- 1 root root 368848687 Dec 10 11:32 'KoBERT_emotion_finetuned_data_added.pt의 사본'
-rw------- 1 root root 368853165 Dec 11 07:53  KoBERT_emotion_finetuned_data_j

#KoBERT 다운로드(사용)#

In [6]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-i2g7f96b
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-i2g7f96b


In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [8]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [9]:
#GPU 사용
device = torch.device("cuda:0")

In [10]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


#실행 환경(사용)#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [11]:
!pip install -U torchtext==0.6.0
!pip install transformers

In [12]:
#!pip install mxnet
#!pip install gluonnlp pandas tqdm
#!pip install sentencepiece
#!pip install transformers==3.0.2
#!pip install torch

# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import torch

# Preliminaries

from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

# Models

import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [13]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#KoBERT 입력 데이터로 변환(사용)#

In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [15]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#추출한 데이터에 감정 레이블 추가(사용)#

In [16]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

새로운 문장을 테스트 할 때, 입력되는 문장을 KoBERT의 입력 형식으로 바꿔주는 코드를 작성해주어야 한다. 아래 코드를 작성하여 토큰화, 패딩, 텐서를 바꿔주고 예측을 하는 'predict' 함수를 만들어 주었다.

In [17]:
model1 = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model1.load_state_dict(torch.load(SAVEPOINT_PATH), strict=False)
model1.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [18]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

dic = {0:'공포', 1:'놀람', 2:'분노', 3:'슬픔', 4:'중립', 5:'행복', 6:'혐오'}
dic = {0:'fear', 1:'surprise', 2:'anger', 3:'sadness', 4:'neutral', 5:'happiness', 6:'disgust'}
r_dic = {'fear':0, 'surprise':1, 'anger':2, 'sadness':3, 'neutral':4, 'happiness':5, 'disgust':6}
def getEmotion(out):
  idx_eval=[]
  val_eval=[]
  for i in out:
    logits=i
    logits = logits.detach().cpu().numpy()
    idx = np.argmax(logits)
    idx_eval.append(idx)
    val_eval.append(dic.get(idx))
    return idx_eval[0], val_eval[0]


def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)

        return getEmotion(out)


        # file wrting

using cached model


#Eval

In [19]:
r_dic.get('sadness')

3

In [20]:
import csv

y = []
p = []
def Eval(filename):
  with open(filename, 'r') as f:
    csv_data = csv.reader(f, delimiter='|')
    next(csv_data)
    for i, line in enumerate(csv_data):
      text = line[0]
      label = int(line[1])
      label_idx = label
      #label_idx = r_dic.get(label)
      idx, val = predict(text)
      #print('(%s/%s) %s\n' % (val, label, text))
      print('[%d](%c)(%d/%d) %s\n' % (i, 'O' if (idx == label_idx) else 'X', idx, label_idx, text))
      p.append(idx)
      y.append(label_idx)
      if(i == 60000):
        return
  #print('>> Generated file %s/%s.answer\n' % (path, jsonfilename))

  #with open('%s/%s.data' % (path, jsonfilename), 'w') as f:
  #  for u in ut:
  #    f.write('%s\t%s\n' % (u['standard_form'], u['dialect_form']))


In [21]:
filename = '/content/drive/MyDrive/KAIST/SEP531/std_test.csv'

In [ ]:
Eval(filename)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[0](O)(4/4) 요즘도 요즘 뭐~

[1](O)(3/3) 코로나 터지고는 헬스장 운영 안 해 가지고 한 별로 안 가고 못 가고 이랬는데

[2](O)(4/4) 그래도 또 하면 조금 약간 스트레스 풀리더라고 그래서

[3](O)(4/4) -꾸준- 꾸준히 할까 생각 중이기도 하고

[4](O)(0/0) 근데 그거 하면은 다리 알 생기고 그런 거 아니지?

[5](O)(4/4) 아~ 그런 거 아니야 그냥 확실히 다리하고 다릿 살은 허벅지살 특히 많이 빠져.

[6](O)(4/4) 그런 거 하면 좋드라 그러니까

[7](O)(3/3) 하면 확실히 여자분 들은 이~ 다리 지방 많은 그~ 다리 이쪽이 많이 빠지드라 (()) 허벅지

[8](O)(3/3) 힘이 많이 들어가서 많이 다리는 조금 빠지는 거 같

[9](O)(4/4) 다리나 엉덩이 뒤쪽이랄까.

[10](O)(6/6) 아니 나는 너무 알이 잘 생겨 가지고 난

[11](O)(3/3) 아~ 난 일반적인 여자가 너무 부러워.

[12](O)(4/4) 아니 그러니까 나는 전체적으로 약간 나는 그러니까

[13](O)(3/3) 남자 같으면은 가슴이 안 나와야 되잖아.

[14](O)(4/4) 가슴도 있는데 몸에 남자같이 막 털이 많고 있잖아.

[15](O)(6/6) 알이 조금 잘 생겨 그러니까 사람들이 그러잖아 아~ 여자들이 알

[16](O)(1/1) 잘 안 생기니까 막 헬스하는데 앓는 소리 하지 마세요 하는데 나는 진짜 일반적인 범주를 넘어서 좀 많거든?

[17](O)(4/4) 나는 전에 수영을 했어.

[18](O)(1/1) 그~ 아는 언니가 수영하면은 다리 살은 쫙 빠지는데 더 붙일 필요 없잖아?

[19](O)(4/4) 그래 가지고 우선 수영을 했다.

[20](O)(3/3) 수영을 하면은 할 때 너무 좋은데 하고 나서 진짜 나 그런 극도의 배고픔은 처음인 (()) 아니 그~

[21](O)(3/3) 물에서는 노는 물놀이해도 엄청 배고프잖아.

[22](O)(1/1) 나오니까 진 장난 아니더라 (()

#F1 score

In [ ]:
print(metrics.precision_score(y, p, average=None)) #
print(metrics.precision_score(y, p, average=None).mean()) #
print(metrics.precision_score(y, p, average='macro')) #1.0
print(metrics.precision_score(y, p, average='micro')) #1.0

[0.93203883 0.         0.88199276 0.75765625 0.00183688 0.
 0.87878788]
0.4931875160128841
0.4931875160128841
0.6429549500205395


In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import roc_auc_score

# roc_auc_score
print('auc', roc_auc_score(y,p))
# sklearn 을 이용하면 전부 계산해준다.
print('accuracy', metrics.accuracy_score(y,p) )
print('precision1', metrics.precision_score(y, p, average=None)) #
print('precision2', metrics.precision_score(y, p, average=None).mean()) #
print('precision3', metrics.precision_score(y, p, average='macro')) #1.0
print('precision4', metrics.precision_score(y, p, average='micro')) #1.0
print('recall1', metrics.recall_score(y, p, average=None)) #
print('recall2', metrics.recall_score(y, p, average=None).mean()) #
print('recall3', metrics.recall_score(y, p, average='macro')) #1.0
print('recall4', metrics.recall_score(y, p, average='micro')) #1.0
print('f1_score1', metrics.f1_score(y, p, average=None)) #
print('f1_score2', metrics.f1_score(y, p, average=None).mean()) #
print('f1_score3', metrics.f1_score(y, p, average='macro')) #1.0
print('f1_score4', metrics.f1_score(y, p, average='micro')) #1.0

print(metrics.classification_report(y,p))
print(metrics.confusion_matrix(y,p))



accuracy 0.6429549500205395
precision1 [0.93203883 0.         0.88199276 0.75765625 0.00183688 0.
 0.87878788]
precision2 0.4931875160128841
precision3 0.4931875160128841
precision4 0.6429549500205395
recall1 [0.42504744 0.         0.60790332 0.77066116 1.         0.
 0.45940594]
recall2 0.4661454077965049
recall3 0.4661454077965049
recall4 0.6429549500205395
f1_score1 [0.58384014 0.         0.71973654 0.76410337 0.00366703 0.
 0.60338101]
f1_score2 0.38210401459325277
f1_score3 0.38210401459325277
f1_score4 0.6429549500205395
              precision    recall  f1-score   support

           0       0.93      0.43      0.58      1581
           1       0.00      0.00      0.00         0
           2       0.88      0.61      0.72      5213
           3       0.76      0.77      0.76      6292
           4       0.00      1.00      0.00         5
           5       0.00      0.00      0.00         0
           6       0.88      0.46      0.60      1515

    accuracy                     

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill